In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from stable_diffusion_backend.ml.image_generation import text_to_img, initialize_model

# scheduler = None
scheduler = "DPM++ SDE Karras"
# scheduler = "DPM++ 2M Karras"

model = initialize_model(scheduler=scheduler)

/home/ilker/Documents/MyRepos/stable-diffusion-backend/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
prompt = "ultra detailed 8k cg, japanese ramen, chopsticks, egg, steam,"

negative_prompt = None
# negative_prompt = "render, cartoon, cgi, render, illustration, painting, drawing"
# negative_prompt = "cgi, 3d render, bad quality, worst quality, text, signature, watermark, extra limbs"

seed = None
# seed = 2022514664

clip_skip = None
# clip_skip = 1
# clip_skip = 2

image = text_to_img(
    model=model,
    prompt=prompt,
    negative_prompt=negative_prompt,
    seed=seed,
    num_inference_steps=7,
    cfg_scale=3.0,
    clip_skip=clip_skip,
)

image

## Intermediate Images

- [Diffusers Issue](https://github.com/huggingface/diffusers/issues/3579)
